<a href="https://colab.research.google.com/github/natanrajch/subte_flow/blob/main/movimiento_pasajeros_subte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Lectura datos

In [5]:
!pip install openpyxl --upgrade

In [6]:
import pandas as pd

In [7]:
df = pd.read_excel("molinetes090821.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279216 entries, 0 to 279215
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   N°Tarjeta  279216 non-null  int64         
 1   Linea      279216 non-null  object        
 2   Estacion   279216 non-null  object        
 3   Fecha      279216 non-null  datetime64[ns]
 4   Hora       279216 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 10.7+ MB


##Transformación de datos

In [8]:
from sklearn.preprocessing import LabelEncoder
df["id"] = LabelEncoder().fit_transform(df['N°Tarjeta'])
df.drop("N°Tarjeta",axis=1,inplace=True)
df.head()

,Linea,Estacion,Fecha,Hora,id
0,Linea D,Catedral,2021-08-09,05:33:00,105381
1,Line B,JMRosas,2021-08-09,05:44:00,7464
2,Line B,Pellegrini,2021-08-09,05:45:00,12425
3,Line B,JMRosas,2021-08-09,05:45:00,12434
4,Line B,Pellegrini,2021-08-09,05:46:00,54433


In [9]:
df["nombre_completo"] = df['Linea'] + df['Estacion']
df.sort_values(["id","Fecha","Hora"], inplace=True)
df.head(5)

,Linea,Estacion,Fecha,Hora,id,nombre_completo
155542,Line C,Retiro,2021-08-09,14:38:00,0,Line C Retiro
199832,Line C,Constitucion,2021-08-09,16:43:00,1,Line C Constitucion
67199,Line B,Lacroze,2021-08-09,09:42:00,2,Line B Lacroze
172974,Line B,Alem,2021-08-09,15:33:00,2,Line B Alem
77994,Linea D,Congreso_Tuc,2021-08-09,10:20:00,3,Linea D Congreso_Tuc


In [10]:
#Las entradas en Línea P Estación Metrovías se convierten en Línea E - Virreyes. Esto es una fuente posible de error, pero menor. Con los datos revisados, tiene más sentido tomar esto que eliminar las fichadas de Line P

df["nombre_completo"] = df.nombre_completo.replace({"Linea P   METROVIAS           ":"Line E    Virreyes            "})
len(df[df.Estacion == "METROVIAS           "])

1268

In [11]:
#Modificar nombres para incorporar Combinaciones y el caso Pasco/Alberti -- "REVISAR CASO PASCO ALBERTI QUE NO ESTÁ BIEN REPRESENTADO EN EL DICT ""normalizar estaciones"" "
normalizar_combinaciones = {"Line A    Lima                ":"Combinación A-C","Line A    Miserere            ":"Combinación A-H","Line A    Peru                ":"Combinación A-D-E","Line B    Alem                ":"Combinación B-E","Line B    Pellegrini          ":"Combinación B-C-D","Line B    Pueyrredon_B        ":"Combinación B-H","Line C    Av. de Mayo         ":"Combinación A-C","Line C    Diagonal_Norte      ":"Combinación B-C-D","Line C    Independencia_C     ":"Combinación C-E Ind","Line C    Retiro              ":"Combinación C-E Ret","Line E    Bolivar             ":"Combinación A-D-E","Line E    Correo Central      ":"Combinación B-E","Line E    Independencia_E     ":"Combinación C-E Ind","Line E    Jujuy               ":"Combinación E-H","Line E    RetiroE             ":"Combinación C-E Ret","Linea D   9 de Julio          ":"Combinación B-C-D","Linea D   Catedral            ":"Combinación A-D-E","Linea D   Pueyrredon_D        ":"Combinación D-H","Linea H   Corrientes          ":"Combinación B-H","Linea H   Humberto Primo      ":"Combinación E-H","Linea H   Once                ":"Combinación A-H","Linea H   Santa Fe            ":"Combinación D-H","Line A    Pasco":"Line A Pasco-Alberti","Line A    Alberti":"Line A Pasco-Alberti"}
df["estac_y_combins"] = df.nombre_completo.replace(normalizar_combinaciones)
print("REVISAR CASO PASCO ALBERTI QUE NO ESTÁ BIEN REPRESENTADO EN EL DICT ""normalizar combinaciones"" ","\n","\n")
df.head(2)

REVISAR CASO PASCO ALBERTI QUE NO ESTÁ BIEN REPRESENTADO EN EL DICT normalizar combinaciones  
 



,Linea,Estacion,Fecha,Hora,id,nombre_completo,estac_y_combins
155542,Line C,Retiro,2021-08-09,14:38:00,0,Line C Retiro,Combinación C-E Ret
199832,Line C,Constitucion,2021-08-09,16:43:00,1,Line C Constitucion,Line C Constitucion


In [12]:
#Eliminar fichadas repetidas en misma estación
fichada_repetida = ((df.estac_y_combins.shift(-1) == df.estac_y_combins) & (df.id.shift(-1)==df.id))
df["fichada_repetida"] = fichada_repetida
df_norepeat = df[~fichada_repetida]
df_norepeat

,Linea,Estacion,Fecha,Hora,id,nombre_completo,estac_y_combins,fichada_repetida
155542,Line C,Retiro,2021-08-09,14:38:00,0,Line C Retiro,Combinación C-E Ret,False
199832,Line C,Constitucion,2021-08-09,16:43:00,1,Line C Constitucion,Line C Constitucion,False
67199,Line B,Lacroze,2021-08-09,09:42:00,2,Line B Lacroze,Line B Lacroze,False
172974,Line B,Alem,2021-08-09,15:33:00,2,Line B Alem,Combinación B-E,False
77994,Linea D,Congreso_Tuc,2021-08-09,10:20:00,3,Linea D Congreso_Tuc,Linea D Congreso_Tuc,False
...,...,...,...,...,...,...,...,...
61936,Line E,Urquiza,2021-08-09,09:26:00,174850,Line E Urquiza,Line E Urquiza,False
139699,Line C,Constitucion,2021-08-09,13:49:00,174851,Line C Constitucion,Line C Constitucion,False
198499,Linea D,Facultad,2021-08-09,16:39:00,174851,Linea D Facultad,Linea D Facultad,False
91475,Linea H,Once,2021-08-09,11:15:00,174852,Linea H Once,Combinación A-H,False


In [13]:
df_norepeat.id.value_counts()[0]

1

In [14]:
#Eliminar tarjetas con 1 sola transacción
conteo_fichadas = df_norepeat.id.map(df_norepeat.id.value_counts())
df2 = df_norepeat[conteo_fichadas > 1].copy() #Solo Tarjetas con al menos 2 fichadas
df2.drop("fichada_repetida", axis=1, inplace=True)
df2.head(2)


,Linea,Estacion,Fecha,Hora,id,nombre_completo,estac_y_combins
67199,Line B,Lacroze,2021-08-09,09:42:00,2,Line B Lacroze,Line B Lacroze
172974,Line B,Alem,2021-08-09,15:33:00,2,Line B Alem,Combinación B-E


In [15]:
primer_ingreso = df2.groupby("id").first().nombre_completo
df2["primer_ingreso"] = df2.id.map(primer_ingreso)
df2.head(5)

,Linea,Estacion,Fecha,Hora,id,nombre_completo,estac_y_combins,primer_ingreso
67199,Line B,Lacroze,2021-08-09,09:42:00,2,Line B Lacroze,Line B Lacroze,Line B Lacroze
172974,Line B,Alem,2021-08-09,15:33:00,2,Line B Alem,Combinación B-E,Line B Lacroze
77994,Linea D,Congreso_Tuc,2021-08-09,10:20:00,3,Linea D Congreso_Tuc,Linea D Congreso_Tuc,Linea D Congreso_Tuc
141896,Linea D,Pueyrredon_D,2021-08-09,13:56:00,3,Linea D Pueyrredon_D,Combinación D-H,Linea D Congreso_Tuc
26574,Line A,Primera Junta,2021-08-09,07:51:00,4,Line A Primera Junta,Line A Primera Junta,Line A Primera Junta


In [16]:
df2["origen"] = df2.nombre_completo
df2["destino"] = df2.groupby("id").nombre_completo.shift(-1)
df2.destino.fillna(df2.primer_ingreso, inplace=True)
df2.head(6)

,Linea,Estacion,Fecha,Hora,id,nombre_completo,estac_y_combins,primer_ingreso,origen,destino
67199,Line B,Lacroze,2021-08-09,09:42:00,2,Line B Lacroze,Line B Lacroze,Line B Lacroze,Line B Lacroze,Line B Alem
172974,Line B,Alem,2021-08-09,15:33:00,2,Line B Alem,Combinación B-E,Line B Lacroze,Line B Alem,Line B Lacroze
77994,Linea D,Congreso_Tuc,2021-08-09,10:20:00,3,Linea D Congreso_Tuc,Linea D Congreso_Tuc,Linea D Congreso_Tuc,Linea D Congreso_Tuc,Linea D Pueyrredon_D
141896,Linea D,Pueyrredon_D,2021-08-09,13:56:00,3,Linea D Pueyrredon_D,Combinación D-H,Linea D Congreso_Tuc,Linea D Pueyrredon_D,Linea D Congreso_Tuc
26574,Line A,Primera Junta,2021-08-09,07:51:00,4,Line A Primera Junta,Line A Primera Junta,Line A Primera Junta,Line A Primera Junta,Line A Peru
230802,Line A,Peru,2021-08-09,17:48:00,4,Line A Peru,Combinación A-D-E,Line A Primera Junta,Line A Peru,Line A Primera Junta


In [17]:
#Chequear que entre primer ingreso y último ingreso no sea la misma estación o nodo de combinación
check_vuelta_repetida = df2.origen.replace(normalizar_combinaciones) == df2.destino.replace(normalizar_combinaciones)
df3 = df2[~check_vuelta_repetida].copy()
df3.head(2)

,Linea,Estacion,Fecha,Hora,id,nombre_completo,estac_y_combins,primer_ingreso,origen,destino
67199,Line B,Lacroze,2021-08-09,09:42:00,2,Line B Lacroze,Line B Lacroze,Line B Lacroze,Line B Lacroze,Line B Alem
172974,Line B,Alem,2021-08-09,15:33:00,2,Line B Alem,Combinación B-E,Line B Lacroze,Line B Alem,Line B Lacroze


In [18]:
df3[(df3.id>700)&(df.id<710)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Linea,Estacion,Fecha,Hora,id,nombre_completo,estac_y_combins,primer_ingreso,origen,destino
33422,Line A,Primera Junta,2021-08-09,08:11:00,701,Line A Primera Junta,Line A Primera Junta,Line A Primera Junta,Line A Primera Junta,Line A SanPedrito
214105,Line A,SanPedrito,2021-08-09,17:13:00,701,Line A SanPedrito,Line A SanPedrito,Line A Primera Junta,Line A SanPedrito,Line A Primera Junta
43745,Line B,Los Incas,2021-08-09,08:36:00,702,Line B Los Incas,Line B Los Incas,Line B Los Incas,Line B Los Incas,Line B Malabia
171736,Line B,Malabia,2021-08-09,15:29:00,702,Line B Malabia,Line B Malabia,Line B Los Incas,Line B Malabia,Line B Los Incas
71562,Linea D,Tribunales,2021-08-09,09:56:00,707,Linea D Tribunales,Linea D Tribunales,Linea D Tribunales,Linea D Tribunales,Linea D Juramento
190583,Linea D,Juramento,2021-08-09,16:20:00,707,Linea D Juramento,Linea D Juramento,Linea D Tribunales,Linea D Juramento,Line B Pueyrredon_B
226121,Line B,Pueyrredon_B,2021-08-09,17:37:00,707,Line B Pueyrredon_B,Combinación B-H,Linea D Tribunales,Line B Pueyrredon_B,Linea D Tribunales
138442,Linea D,Juramento,2021-08-09,13:44:00,708,Linea D Juramento,Linea D Juramento,Linea D Juramento,Linea D Juramento,Linea D Pueyrredon_D
202370,Linea D,Pueyrredon_D,2021-08-09,16:49:00,708,Linea D Pueyrredon_D,Combinación D-H,Linea D Juramento,Linea D Pueyrredon_D,Linea D Juramento
51456,Line B,JMRosas,2021-08-09,08:56:00,709,Line B JMRosas,Line B JMRosas,Line B JMRosas,Line B JMRosas,Line B Callao_B


In [19]:
len(df), len(df2), len(df3), len(df3) / len(df)

(279216, 175807, 174886, 0.6263466277004183)

In [20]:
normalizar_nombres = {"Line A    Acoyte              ":"Acoyte","Line A    Carabobo            ":"Carabobo","Line A    Castro Barros       ":"Castro Barros","Line A    Congreso            ":"Congreso","Line A    Flores              ":"San José de Flores","Line A    Lima                ":"Lima","Line A    Loria               ":"Loria","Line A    Miserere            ":"Plaza Miserere","Line A    Peru                ":"Perú","Line A    Plaza de Mayo       ":"Plaza de Mayo","Line A    Primera Junta       ":"Primera Junta","Line A    Puan                ":"Puán","Line A    SanPedrito          ":"San Pedrito","Line B    Alem                ":"Leandro N. Alem","Line B    Callao_B            ":"Callao B","Line B    Dorrego             ":"Dorrego","Line B    Gallardo            ":"Angel Gallardo","Line B    Gardel              ":"Carlos Gardel","Line B    JMRosas             ":"Juan Manuel de Rosas","Line B    Lacroze             ":"Federico Lacroze","Line B    Los Incas           ":"Los Incas","Line B    Malabia             ":"Malabia","Line B    Medrano             ":"Medrano","Line B    Pellegrini          ":"Carlos Pellegrini","Line B    Pueyrredon_B        ":"Pueyrredón B","Line B    Uruguay             ":"Uruguay","Line C    Av. de Mayo         ":"Avenida de Mayo","Line C    Constitucion        ":"Constitución","Line C    Diagonal_Norte      ":"Diagonal Norte","Line C    Independencia_C     ":"Independencia C","Line C    Lavalle             ":"Lavalle","Line C    Retiro              ":"Retiro C","Line C    San Martin          ":"General San Martín","Line E    Boedo               ":"Boedo","Line E    Bolivar             ":"Bolívar","Line E    Correo Central      ":"Correo Central","Line E    E Mitre             ":"Emilio Mitre","Line E    Entre Rio           ":"Entre Ríos","Line E    Independencia_E     ":"Independencia E","Line E    Jujuy               ":"Jujuy","Line E    La Plata            ":"La Plata","Line E    Milagrosa           ":"Medalla Milagrosa","Line E    Moreno              ":"José María Moreno","Line E    RetiroE             ":"Retiro E","Line E    Urquiza             ":"General Urquiza","Line E    Virreyes            ":"Plaza de los Virreyes","Linea D   9 de Julio          ":"9 de Julio","Linea D   Bulnes              ":"Bulnes","Linea D   Callao_D            ":"Callao D","Linea D   Carranza            ":"Ministro Carranza","Linea D   Catedral            ":"Catedral","Linea D   Congreso_Tuc        ":"Congreso de Tucumán","Linea D   Facultad            ":"Facultad de Medicina","Linea D   Juramento           ":"Juramento","Linea D   Olleros             ":"Olleros","Linea D   Palermo             ":"Palermo","Linea D   Plaza Italia        ":"Plaza Italia","Linea D   Pueyrredon_D        ":"Pueyrredón D","Linea D   Scalabrini Ortiz    ":"Scalabrini Ortiz","Linea D   Tribunales          ":"Tribunales","Linea H   Caseros             ":"Caseros","Linea H   Corrientes          ":"Corrientes","Linea H   FacultadDerecho     ":"Facultad de Derecho","Linea H   Hospitales          ":"Hospitales","Linea H   Humberto Primo      ":"Humberto 1","Linea H   LasHeras            ":"Las Heras","Linea H   Once                ":"Once","Linea H   Patricios           ":"Parque Patricios","Linea H   Santa Fe            ":"Santa Fe","Linea H   Venezuela           ":"Venezuela"}

df3["origen_norm"] = df3.origen.replace(normalizar_nombres)
df3["destino_norm"] = df3.destino.replace(normalizar_nombres)
df3.head(3)

,Linea,Estacion,Fecha,Hora,id,nombre_completo,estac_y_combins,primer_ingreso,origen,destino,origen_norm,destino_norm
67199,Line B,Lacroze,2021-08-09,09:42:00,2,Line B Lacroze,Line B Lacroze,Line B Lacroze,Line B Lacroze,Line B Alem,Federico Lacroze,Leandro N. Alem
172974,Line B,Alem,2021-08-09,15:33:00,2,Line B Alem,Combinación B-E,Line B Lacroze,Line B Alem,Line B Lacroze,Leandro N. Alem,Federico Lacroze
77994,Linea D,Congreso_Tuc,2021-08-09,10:20:00,3,Linea D Congreso_Tuc,Linea D Congreso_Tuc,Linea D Congreso_Tuc,Linea D Congreso_Tuc,Linea D Pueyrredon_D,Congreso de Tucumán,Pueyrredón D


In [21]:
# minute_bins = [-1,15,30,45,60]
# labels = ["00-15","16-30","31-45","46-59"]
# minute_bins = [-1,20,40,60]
# labels = ["00-20","21-39","40-59"]
minute_bins = [-1,30,60]
labels = ["00-30","31-59"]

df3["time_bins"] = df3.Hora.apply(lambda hora: "h:" + str(hora.hour) + ":m:") + pd.cut(df3.Hora.apply(lambda hora: hora.minute), bins=minute_bins, labels=labels).astype(str)
df3.head()

,Linea,Estacion,Fecha,Hora,id,nombre_completo,estac_y_combins,primer_ingreso,origen,destino,origen_norm,destino_norm,time_bins
67199,Line B,Lacroze,2021-08-09,09:42:00,2,Line B Lacroze,Line B Lacroze,Line B Lacroze,Line B Lacroze,Line B Alem,Federico Lacroze,Leandro N. Alem,h:9:m:31-59
172974,Line B,Alem,2021-08-09,15:33:00,2,Line B Alem,Combinación B-E,Line B Lacroze,Line B Alem,Line B Lacroze,Leandro N. Alem,Federico Lacroze,h:15:m:31-59
77994,Linea D,Congreso_Tuc,2021-08-09,10:20:00,3,Linea D Congreso_Tuc,Linea D Congreso_Tuc,Linea D Congreso_Tuc,Linea D Congreso_Tuc,Linea D Pueyrredon_D,Congreso de Tucumán,Pueyrredón D,h:10:m:00-30
141896,Linea D,Pueyrredon_D,2021-08-09,13:56:00,3,Linea D Pueyrredon_D,Combinación D-H,Linea D Congreso_Tuc,Linea D Pueyrredon_D,Linea D Congreso_Tuc,Pueyrredón D,Congreso de Tucumán,h:13:m:31-59
26574,Line A,Primera Junta,2021-08-09,07:51:00,4,Line A Primera Junta,Line A Primera Junta,Line A Primera Junta,Line A Primera Junta,Line A Peru,Primera Junta,Perú,h:7:m:31-59


In [22]:
#Chequear si todas las estaciones+time bins de df1 existen en df3
df["time_bins"] = df.Hora.apply(lambda hora: "h:" + str(hora.hour) + ":m:") + pd.cut(df.Hora.apply(lambda hora: hora.minute), bins=minute_bins, labels=labels).astype(str)

validaciones_estacion_timebin_df1 = df.groupby(["nombre_completo","time_bins"]).nombre_completo.count()
validaciones_estacion_timebin_df3 = df3.groupby(["nombre_completo","time_bins"]).nombre_completo.count()

estaciones_time_bins = df3.groupby(["nombre_completo","time_bins"]).destino.count().index
total = 0
for bin in df.groupby(["nombre_completo","time_bins"]).nombre_completo.count().index:
  if bin not in estaciones_time_bins:
    print(bin, validaciones_estacion_timebin_df1[bin])
    total += validaciones_estacion_timebin_df1[bin]
print("Casos no registrados:",total) 
#('Line A    Acoyte              ', 'h:10:m:00-14') in df3.groupby(["nombre_completo","time_bins"]).destino.count().index
#list(df3.groupby(["nombre_completo","time_bins","destino"]).destino.count().index[0])[:2]

('Line A    Castro Barros       ', 'h:21:m:31-59') 1
('Line A    Plaza de Mayo       ', 'h:21:m:31-59') 1
('Line B    Dorrego             ', 'h:5:m:31-59') 1
('Line B    Gallardo            ', 'h:21:m:31-59') 1
('Line B    Lacroze             ', 'h:21:m:31-59') 1
('Line B    Tronador            ', 'h:17:m:00-30') 1
('Line C    Independencia_C     ', 'h:22:m:00-30') 1
('Line C    Lavalle             ', 'h:21:m:31-59') 1
('Line C    San Martin          ', 'h:11:m:00-30') 1
('Line C    San Martin          ', 'h:5:m:31-59') 1
('Line E    La Plata            ', 'h:21:m:31-59') 1
('Line E    Milagrosa           ', 'h:21:m:00-30') 2
('Line E    Virreyes            ', 'h:21:m:31-59') 3
('Linea D   9 de Julio          ', 'h:22:m:31-59') 1
('Linea D   Congreso_Tuc        ', 'h:22:m:00-30') 1
('Linea D   Facultad            ', 'h:5:m:31-59') 1
('Linea H   FacultadDerecho     ', 'h:22:m:00-30') 1
('Linea H   FacultadDerecho     ', 'h:5:m:31-59') 1
('Linea H   Humberto Primo      ', 'h:21:m:31-59')

In [23]:
#Crear en df3 una columna con el proporcional de la validación conocida con origen destino al total de validaciones en df
#validaciones_estacion_timebin_destino_df3 = df3.groupby(["nombre_completo","time_bins","destino"]).nombre_completo.count()

df3["validaciones"] = df3.apply(lambda row: validaciones_estacion_timebin_df1[(row.nombre_completo, row.time_bins)] / validaciones_estacion_timebin_df3[(row.nombre_completo, row.time_bins)], axis=1 )
df3.head()


,Linea,Estacion,Fecha,Hora,id,nombre_completo,estac_y_combins,primer_ingreso,origen,destino,origen_norm,destino_norm,time_bins,validaciones
67199,Line B,Lacroze,2021-08-09,09:42:00,2,Line B Lacroze,Line B Lacroze,Line B Lacroze,Line B Lacroze,Line B Alem,Federico Lacroze,Leandro N. Alem,h:9:m:31-59,1.316418
172974,Line B,Alem,2021-08-09,15:33:00,2,Line B Alem,Combinación B-E,Line B Lacroze,Line B Alem,Line B Lacroze,Leandro N. Alem,Federico Lacroze,h:15:m:31-59,1.772059
77994,Linea D,Congreso_Tuc,2021-08-09,10:20:00,3,Linea D Congreso_Tuc,Linea D Congreso_Tuc,Linea D Congreso_Tuc,Linea D Congreso_Tuc,Linea D Pueyrredon_D,Congreso de Tucumán,Pueyrredón D,h:10:m:00-30,1.505155
141896,Linea D,Pueyrredon_D,2021-08-09,13:56:00,3,Linea D Pueyrredon_D,Combinación D-H,Linea D Congreso_Tuc,Linea D Pueyrredon_D,Linea D Congreso_Tuc,Pueyrredón D,Congreso de Tucumán,h:13:m:31-59,1.691176
26574,Line A,Primera Junta,2021-08-09,07:51:00,4,Line A Primera Junta,Line A Primera Junta,Line A Primera Junta,Line A Primera Junta,Line A Peru,Primera Junta,Perú,h:7:m:31-59,1.573770


In [24]:
len(df) - sum(df3.validaciones), len(df3)

(22.000000009371433, 174886)

##Cálculo Shortest Path

Edge Data from Subte GTFS
Linea E times between stations simplified into only one value (I guess it was some data noise in original gtfs)
Pasco y Alberti joined into one station
Correo Central, Catalinas and Retiro E manually added to gtfs data


In [25]:
import networkx as nx

In [26]:
nodes = ["Plaza de Mayo","Perú","Piedras","Lima","Sáenz Peña","Congreso","Pasco-Alberti","Plaza Miserere","Loria","Castro Barros","Río de Janeiro","Acoyte","Primera Junta","Puán","Carabobo","San José de Flores","San Pedrito","Leandro N. Alem","Florida","Carlos Pellegrini","Uruguay","Callao B","Pasteur","Pueyrredón B","Carlos Gardel","Medrano","Angel Gallardo","Malabia","Dorrego","Federico Lacroze","Tronador","Los Incas","Echeverría","Juan Manuel de Rosas","Retiro C","General San Martín","Lavalle","Diagonal Norte","Avenida de Mayo","Moreno","Independencia C","San Juan","Constitución","Catedral","9 de Julio","Tribunales","Callao D","Facultad de Medicina","Pueyrredón D","Agüero","Bulnes","Scalabrini Ortiz","Plaza Italia","Palermo","Ministro Carranza","Olleros","José Hernández","Juramento","Congreso de Tucumán","Retiro E","Catalinas","Correo Central","Bolívar","Belgrano","Independencia E","San José","Entre Ríos","Pichincha","Jujuy","General Urquiza","Boedo","La Plata","José María Moreno","Emilio Mitre","Medalla Milagrosa","Varela","Plaza de los Virreyes","Facultad de Derecho","Las Heras","Santa Fe","Córdoba","Corrientes","Once","Venezuela","Humberto 1","Inclán","Caseros","Parque Patricios","Hospitales"]

In [27]:
G = nx.DiGraph()
G.add_nodes_from(nodes)

In [28]:
edges = [["9 de Julio","Carlos Pellegrini",3.1000],["9 de Julio","Diagonal Norte",5.4000],["Avenida de Mayo","Lima",3.1000],["Bolívar","Catedral",5.4000],["Bolívar","Perú",3.4000],["Carlos Pellegrini","9 de Julio",3.1000],["Carlos Pellegrini","Diagonal Norte",6.0000],["Catedral","Bolívar",5.4000],["Catedral","Perú",3.1000],["Corrientes","Pueyrredón B",3.1000],["Diagonal Norte","9 de Julio",5.4000],["Diagonal Norte","Carlos Pellegrini",6.0000],["Humberto 1","Jujuy",3.1000],["Independencia C","Independencia E",3.7000],["Independencia E","Independencia C",3.7000],["Jujuy","Humberto 1",3.1000],["Lima","Avenida de Mayo",3.1000],["Once","Plaza Miserere",3.1000],["Perú","Bolívar",3.4000],["Perú","Catedral",3.1000],["Plaza Miserere","Once",3.1000],["Pueyrredón B","Corrientes",3.1000],["Pueyrredón D","Santa Fe",3.1000],["Santa Fe","Pueyrredón D",3.1000],["Acoyte","Primera Junta",1.5500],["Acoyte","Río de Janeiro",1.7833],["Pasco-Alberti","Plaza Miserere",1.5833],["Carabobo","Puán",1.8833],["Carabobo","San José de Flores",1.9333],["Castro Barros","Loria",1.6500],["Castro Barros","Río de Janeiro",2.0000],["Congreso","Pasco-Alberti",1.5833],["Congreso","Sáenz Peña",1.5333],["Lima","Piedras",1.1500],["Lima","Sáenz Peña",1.1667],["Loria","Castro Barros",1.6500],["Loria","Plaza Miserere",2.2833],["Pasco-Alberti","Congreso",1.5833],["Perú","Piedras",1.1333],["Perú","Plaza de Mayo",0.9667],["Piedras","Lima",1.1500],["Piedras","Perú",1.1333],["Plaza de Mayo","Perú",0.9667],["Plaza Miserere","Loria",2.2833],["Plaza Miserere","Pasco-Alberti",1.5833],["Primera Junta","Acoyte",1.5500],["Primera Junta","Puán",1.9333],["Puán","Carabobo",1.8833],["Puán","Primera Junta",1.9333],["Río de Janeiro","Acoyte",1.7833],["Río de Janeiro","Castro Barros",2.0000],["Sáenz Peña","Congreso",1.5333],["Sáenz Peña","Lima",1.1667],["San José de Flores","Carabobo",1.9333],["San José de Flores","San Pedrito",1.7500],["San Pedrito","San José de Flores",1.7500],["Angel Gallardo","Malabia",1.9333],["Angel Gallardo","Medrano",1.9500],["Callao B","Pasteur",1.4333],["Callao B","Uruguay",1.3167],["Carlos Gardel","Medrano",1.9500],["Carlos Gardel","Pueyrredón B",1.4667],["Carlos Pellegrini","Florida",1.4833],["Carlos Pellegrini","Uruguay",1.3167],["Dorrego","Federico Lacroze",1.7167],["Dorrego","Malabia",2.3000],["Echeverría","Juan Manuel de Rosas",1.6167],["Echeverría","Los Incas",1.9167],["Federico Lacroze","Dorrego",1.7167],["Federico Lacroze","Tronador",2.3500],["Florida","Carlos Pellegrini",1.4833],["Florida","Leandro N. Alem",1.1167],["Juan Manuel de Rosas","Echeverría",1.6167],["Leandro N. Alem","Florida",1.1167],["Los Incas","Echeverría",1.9167],["Los Incas","Tronador",1.6500],["Malabia","Angel Gallardo",1.9333],["Malabia","Dorrego",2.3000],["Medrano","Angel Gallardo",1.9500],["Medrano","Carlos Gardel",1.9500],["Pasteur","Callao B",1.4333],["Pasteur","Pueyrredón B",1.3333],["Pueyrredón B","Carlos Gardel",1.4667],["Pueyrredón B","Pasteur",1.3333],["Tronador","Federico Lacroze",2.3500],["Tronador","Los Incas",1.6500],["Uruguay","Callao B",1.3167],["Uruguay","Carlos Pellegrini",1.3167],["Avenida de Mayo","Diagonal Norte",1.6667],["Avenida de Mayo","Moreno",1.3000],["Constitución","San Juan",1.6500],["Diagonal Norte","Avenida de Mayo",1.6667],["Diagonal Norte","Lavalle",1.3500],["General San Martín","Lavalle",2.2500],["General San Martín","Retiro C",1.3000],["Independencia C","Moreno",1.8167],["Independencia C","San Juan",1.5833],["Lavalle","Diagonal Norte",1.3500],["Lavalle","General San Martín",2.2500],["Moreno","Avenida de Mayo",1.3000],["Moreno","Independencia C",1.8167],["Retiro C","General San Martín",1.3000],["San Juan","Constitución",1.6500],["San Juan","Independencia C",1.5833],["9 de Julio","Catedral",1.6833],["9 de Julio","Tribunales",1.3167],["Agüero","Bulnes",1.4333],["Agüero","Pueyrredón D",1.4000],["Bulnes","Agüero",1.4333],["Bulnes","Scalabrini Ortiz",1.4500],["Callao D","Facultad de Medicina",1.3000],["Callao D","Tribunales",1.9833],["Catedral","9 de Julio",1.6833],["Congreso de Tucumán","Juramento",2.1500],["Facultad de Medicina","Callao D",1.3000],["Facultad de Medicina","Pueyrredón D",2.0000],["José Hernández","Juramento",1.5167],["José Hernández","Olleros",2.0000],["Juramento","Congreso de Tucumán",2.1500],["Juramento","José Hernández",1.5167],["Ministro Carranza","Olleros",2.4167],["Ministro Carranza","Palermo",2.1333],["Olleros","José Hernández",2.0000],["Olleros","Ministro Carranza",2.4167],["Palermo","Ministro Carranza",2.1333],["Palermo","Plaza Italia",1.5000],["Plaza Italia","Palermo",1.5000],["Plaza Italia","Scalabrini Ortiz",1.5667],["Pueyrredón D","Agüero",1.4000],["Pueyrredón D","Facultad de Medicina",2.0000],["Scalabrini Ortiz","Bulnes",1.4500],["Scalabrini Ortiz","Plaza Italia",1.5667],["Tribunales","9 de Julio",1.3167],["Tribunales","Callao D",1.9833],["Belgrano","Bolívar",1.5000],["Belgrano","Independencia E",1.5333],["Boedo","General Urquiza",1.6500],["Boedo","La Plata",2.0167],["Bolívar","Belgrano",1.5000],["Emilio Mitre","José María Moreno",2.1167],["Emilio Mitre","Medalla Milagrosa",2.1500],["Entre Ríos","Pichincha",1.3833],["Entre Ríos","San José",1.5000],["General Urquiza","Boedo",1.6500],["General Urquiza","Jujuy",1.6000],["Independencia E","Belgrano",1.5333],["Independencia E","San José",1.8000],["José María Moreno","Emilio Mitre",2.1167],["José María Moreno","La Plata",1.9333],["Jujuy","General Urquiza",1.6000],["Jujuy","Pichincha",1.4000],["La Plata","Boedo",2.0167],["La Plata","José María Moreno",1.9333],["Medalla Milagrosa","Emilio Mitre",2.1500],["Medalla Milagrosa","Varela",1.9500],["Pichincha","Entre Ríos",1.3833],["Pichincha","Jujuy",1.4000],["Plaza de los Virreyes","Varela",1.3333],["San José","Entre Ríos",1.5000],["San José","Independencia E",1.8000],["Varela","Medalla Milagrosa",1.9500],["Varela","Plaza de los Virreyes",1.3333],["Caseros","Inclán",1.6667],["Caseros","Parque Patricios",2.0167],["Córdoba","Corrientes",1.5000],["Córdoba","Santa Fe",1.2167],["Corrientes","Córdoba",1.5000],["Corrientes","Once",1.5000],["Facultad de Derecho","Las Heras",1.6500],["Hospitales","Parque Patricios",1.8667],["Humberto 1","Inclán",1.7000],["Humberto 1","Venezuela",2.0833],["Inclán","Caseros",1.6667],["Inclán","Humberto 1",1.7000],["Las Heras","Facultad de Derecho",1.6500],["Las Heras","Santa Fe",2.0667],["Once","Corrientes",1.5000],["Once","Venezuela",1.6667],["Parque Patricios","Caseros",2.0167],["Parque Patricios","Hospitales",1.8667],["Santa Fe","Córdoba",1.2167],["Santa Fe","Las Heras",2.0667],["Venezuela","Humberto 1",2.0833],["Venezuela","Once",1.6667],["Bolívar","Correo Central",2.3739],["Correo Central","Bolívar",2.3739],["Catalinas","Correo Central",2.0391],["Correo Central","Catalinas",2.0391],["Retiro E","Catalinas",1.6961],["Catalinas","Retiro E",1.6961],["Retiro E","Retiro C",3.1000],["Retiro C","Retiro E",3.1000],["Correo Central","Leandro N. Alem",3.1000],["Leandro N. Alem","Correo Central",3.1000]]
for edge in edges:
  u,v,weight=edge
  G.add_edge(u, v, weight=weight)

In [29]:
shortest_paths = {}
for node in nodes:
  shortest_paths[node] = nx.shortest_path(G,source=node, weight="weight")


In [30]:
from collections import defaultdict
edges_weights = defaultdict(dict)
for edge in edges:
  u,v,weight=edge
  edges_weights[u][v] = weight

## Combinación Molinetes con Shortest Path para crear un df de movimientos en el Subte

In [35]:
from datetime import timedelta
from datetime import datetime
from datetime import date


tramos = []
horas = []
validaciones = []
sube_baja = []
origenes=[]
destinos=[]
for row in df3.itertuples():
  path = shortest_paths[row.origen_norm][row.destino_norm]
  mins_acum = 0
  for i in range(len(path)-1):
    origen = path[i]
    destino = path[i+1]
    tramos.append(origen + "-" + destino)
    horas.append((datetime.combine(date(100,1,1),row.Hora) + timedelta(minutes=mins_acum)).time())
    validaciones.append(row.validaciones)
    if i == 0:
      sube_baja.append("sube")
    elif i == (len(path) - 2):
      sube_baja.append("baja")
    else:
      sube_baja.append("-")
    origenes.append(origen)
    destinos.append(destino)
    mins_acum += edges_weights[origen][destino]

In [36]:
  len(tramos), len(horas), len(validaciones), len(sube_baja), len(origenes), len(destinos)

(1690542, 1690542, 1690542, 1690542, 1690542, 1690542)

To DO: convertir los datos obtenidos para una porción de los fichajes en el total de fichajes. Idea: Tomar tarjetas con destino conocido cada 20 min de cada estación en todo el horario. Considerar el % que va a cada destino. Luego tomar el total de tarjetas en esos 20 min, y multiplicar proporcionalmente.
El total de tarjetas se puede realizar con los datos de molinetes, aunque sería bueno tomar la otra base que incluye también a los franquiciados.

In [37]:
cargadf = pd.DataFrame(list(zip(tramos, horas, validaciones,sube_baja, origenes, destinos)),
               columns =['tramos', 'horas','pasajeros','sube_baja', 'origen', 'destino'])
cargadf

,tramos,horas,pasajeros,sube_baja,origen,destino
0,Federico Lacroze-Dorrego,09:42:00,1.316418,sube,Federico Lacroze,Dorrego
1,Dorrego-Malabia,09:43:43.002000,1.316418,-,Dorrego,Malabia
2,Malabia-Angel Gallardo,09:46:01.002000,1.316418,-,Malabia,Angel Gallardo
3,Angel Gallardo-Medrano,09:47:57,1.316418,-,Angel Gallardo,Medrano
4,Medrano-Carlos Gardel,09:49:54,1.316418,-,Medrano,Carlos Gardel
...,...,...,...,...,...,...
1690537,Diagonal Norte-Avenida de Mayo,16:49:00,1.588957,-,Diagonal Norte,Avenida de Mayo
1690538,Avenida de Mayo-Moreno,16:50:40.002000,1.588957,-,Avenida de Mayo,Moreno
1690539,Moreno-Independencia C,16:51:58.002000,1.588957,-,Moreno,Independencia C
1690540,Independencia C-San Juan,16:53:47.004000,1.588957,-,Independencia C,San Juan


In [38]:
cargadf.head(20)

,tramos,horas,pasajeros,sube_baja,origen,destino
0,Federico Lacroze-Dorrego,09:42:00,1.316418,sube,Federico Lacroze,Dorrego
1,Dorrego-Malabia,09:43:43.002000,1.316418,-,Dorrego,Malabia
2,Malabia-Angel Gallardo,09:46:01.002000,1.316418,-,Malabia,Angel Gallardo
3,Angel Gallardo-Medrano,09:47:57,1.316418,-,Angel Gallardo,Medrano
4,Medrano-Carlos Gardel,09:49:54,1.316418,-,Medrano,Carlos Gardel
5,Carlos Gardel-Pueyrredón B,09:51:51,1.316418,-,Carlos Gardel,Pueyrredón B
6,Pueyrredón B-Pasteur,09:53:19.002000,1.316418,-,Pueyrredón B,Pasteur
7,Pasteur-Callao B,09:54:39,1.316418,-,Pasteur,Callao B
8,Callao B-Uruguay,09:56:04.998000,1.316418,-,Callao B,Uruguay
9,Uruguay-Carlos Pellegrini,09:57:24,1.316418,-,Uruguay,Carlos Pellegrini


In [39]:
bins = [-1,10,20,30,40,50,60]
labels= [i for i in range(len(bins)-1)]
labels_text= [str(bins[i]+1)+"-"+str(bins[i+1]) for i in range(len(bins)-1)]
cargadf["time_bin_num"] = cargadf.horas.apply(lambda hora: hora.hour * 10) + pd.cut(cargadf.horas.apply(lambda hora: hora.minute),bins=bins, labels=labels).astype(int)
cargadf["time_bin"] = cargadf.horas.apply(lambda hora: str(hora.hour)+":") + pd.cut(cargadf.horas.apply(lambda hora: hora.minute),bins=bins, labels=labels_text).astype(str)
cargadf.head(5)

,tramos,horas,pasajeros,sube_baja,origen,destino,time_bin_num,time_bin
0,Federico Lacroze-Dorrego,09:42:00,1.316418,sube,Federico Lacroze,Dorrego,94,9:41-50
1,Dorrego-Malabia,09:43:43.002000,1.316418,-,Dorrego,Malabia,94,9:41-50
2,Malabia-Angel Gallardo,09:46:01.002000,1.316418,-,Malabia,Angel Gallardo,94,9:41-50
3,Angel Gallardo-Medrano,09:47:57,1.316418,-,Angel Gallardo,Medrano,94,9:41-50
4,Medrano-Carlos Gardel,09:49:54,1.316418,-,Medrano,Carlos Gardel,94,9:41-50


In [40]:
pd.DataFrame(cargadf.tramos.unique().tolist()).to_excel("tramos.xlsx")

In [41]:
type(cargadf.groupby(["time_bin_num","tramos"]).sum())

pandas.core.frame.DataFrame

In [42]:
# pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True)[source]
(pd.pivot_table(cargadf, values="pasajeros", columns="time_bin_num", index="tramos", aggfunc="sum").fillna(0) * 6).to_excel("tramos_y_cargas_por_hora.xlsx")


In [43]:
df[(df.Estacion == "Constitucion        ") & (df.Hora.apply(lambda hora: hora.hour * 10 + hora.minute).astype(int) < 60)]

,Linea,Estacion,Fecha,Hora,id,nombre_completo,estac_y_combins,fichada_repetida,time_bins
